# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [105]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [106]:
#Cities.csv to dataframe

weather_data = "cities.csv"
weather_data = pd.read_csv(weather_data)
weatherdf = pd.DataFrame(weather_data)
#Clean extra column 
clean_weather_df = weatherdf.drop(columns = ["Unnamed: 0"])
clean_weather_df.head(100)

,City,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed,Latitude,Longitude
0,Avarua,56,CK,1593361057,68,71.60,5.82,-21.21,-159.78
1,Labuhan,0,ID,1593361093,83,77.86,7.45,-6.88,112.21
2,Vaini,75,TO,1593361060,94,75.20,4.70,-21.20,-175.20
3,Yellowknife,75,CA,1593361008,51,63.00,10.29,62.46,-114.35
4,Port Alfred,9,ZA,1593361052,69,61.00,8.01,-33.59,26.89
...,...,...,...,...,...,...,...,...,...
95,Codrington,18,AU,1593361477,85,42.73,7.11,-38.27,141.97
96,Ponta do Sol,4,PT,1593361015,72,73.99,8.99,32.67,-17.10
97,Dagana,0,MR,1593361477,28,107.60,7.00,16.52,-15.51
98,Barrow,1,US,1593361006,75,46.40,8.05,71.29,-156.79


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [107]:
#Configure gmaps 
gmaps.configure(api_key = g_key)

In [108]:
#Locations
locations = clean_weather_df[["Latitude", "Longitude"]].astype(float)

humidity = clean_weather_df["Humidity"].astype(float)

In [109]:
#Heatmap layer
fig = gmaps.figure(center = (48.0, 0.0), zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 2, opacity = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [110]:
#Setting my ideal weather conditions - warm, not humid, less cloudy
ideal_conditions_df = clean_weather_df.loc[(clean_weather_df["Max Temp"]>68) & (clean_weather_df["Max Temp"]<95) & (clean_weather_df["Cloudiness"]<70) & (clean_weather_df["Humidity"]<40)]
ideal_conditions_df.head(1000)
#drop null values 
clean_ideal_conditions = ideal_conditions_df.dropna()
clean_ideal_conditions

,City,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed,Latitude,Longitude
21,Inhambane,0,MZ,1593361058,30,82.40,10.11,-23.86,35.38
26,Cabrobó,0,BR,1593361470,33,87.60,12.80,-8.51,-39.31
45,Yumen,0,CN,1593361069,27,72.16,13.42,40.28,97.20
50,Ivanivka,0,UA,1593361386,38,85.21,6.85,46.72,34.55
85,Matucana,10,PE,1593361476,25,68.65,5.19,-11.85,-76.40
110,Saint George,1,US,1593361081,18,91.40,16.11,37.10,-113.58
159,Safford,1,US,1593361484,30,86.00,9.17,32.83,-109.71
193,Storforshei,5,NO,1593361488,34,80.01,4.70,66.40,14.52
198,Odesa,4,UA,1593361280,33,87.80,13.42,46.48,30.73
241,La Rioja,52,ES,1593360815,31,89.60,5.82,42.25,-2.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [116]:
#Hotel DF and add column 
hotel_df = pd.DataFrame(clean_ideal_conditions[['City', 'Country', 'Latitude', 'Longitude']])
hotel_df.head(10)

,City,Country,Latitude,Longitude
21,Inhambane,MZ,-23.86,35.38
26,Cabrobó,BR,-8.51,-39.31
45,Yumen,CN,40.28,97.20
50,Ivanivka,UA,46.72,34.55
85,Matucana,PE,-11.85,-76.40
110,Saint George,US,37.10,-113.58
159,Safford,US,32.83,-109.71
193,Storforshei,NO,66.40,14.52
198,Odesa,UA,46.48,30.73
241,La Rioja,ES,42.25,-2.50


In [120]:
#Parameters
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#hotels = []

#params = {"type": "hotel","keyword": "hotel", "radius": 5000, "key": g_key}

#for index, row in hotel_df.iterrows():
 #   lat = row["Latitude"]
  #  lng = row["Longitude"]
   # city = row["City"]
    
   # params["location"] = f"{lat},{lng}"
    #response = requests.get(base_url, params = params).json()
    
   # results = response['results']
    
 #   try: 
 #       hotel_df.loc[index, "Hotel Name"] = results[0]['name']
 #   except (KeyError, IndexError):
  #      print("No hotel found")

for index, row in hotel_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    city = row['City']
    country = row['Country']
    
    print(f"Retriving results for {city}")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    results = response['results']
    
    
    try: 
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} is {hotelname}")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname 
    except (KeyError, IndexError):
              print("No hotel found")
    

Retriving results for Inhambane
Closest hotel to Inhambane is Saboeira
Retriving results for Cabrobó
Closest hotel to Cabrobó is Salão do Reino das Testemunhas de Jeová
Retriving results for Yumen
No hotel found
Retriving results for Ivanivka
Closest hotel to Ivanivka is Ivanivsʹka Rayonna Derzhavna Administratsiya Khersonsʹkoyi Oblasti
Retriving results for Matucana
Closest hotel to Matucana is Huariquiña: Turismo, Aventuras y Exploración
Retriving results for Saint George
Closest hotel to Saint George is Fairfield Inn by Marriott St. George
Retriving results for Safford
Closest hotel to Safford is Subway
Retriving results for Storforshei
Closest hotel to Storforshei is Fjellklippen Frisør AS
Retriving results for Odesa
Closest hotel to Odesa is Palladium
Retriving results for La Rioja
Closest hotel to La Rioja is Molino del Corregidor
Retriving results for Saint Lawrence
Closest hotel to Saint Lawrence is St Patricks Hotel Gozo
Retriving results for Dāmghān
Closest hotel to Dāmghān i

In [126]:
hotel_df.head(100)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(100)

,City,Country,Latitude,Longitude,Hotel Name
21,Inhambane,MZ,-23.86,35.38,Saboeira
26,Cabrobó,BR,-8.51,-39.31,Salão do Reino das Testemunhas de Jeová
50,Ivanivka,UA,46.72,34.55,Ivanivsʹka Rayonna Derzhavna Administratsiya K...
85,Matucana,PE,-11.85,-76.40,"Huariquiña: Turismo, Aventuras y Exploración"
110,Saint George,US,37.10,-113.58,Fairfield Inn by Marriott St. George
159,Safford,US,32.83,-109.71,Subway
193,Storforshei,NO,66.40,14.52,Fjellklippen Frisør AS
198,Odesa,UA,46.48,30.73,Palladium
241,La Rioja,ES,42.25,-2.50,Molino del Corregidor
256,Saint Lawrence,MT,36.06,14.20,St Patricks Hotel Gozo


In [127]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [130]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig
plt.savefig("vacationmapwithhotels.png")

Figure(layout=FigureLayout(height='420px'))